**Avant de lancer ce notebook merci de générer les csv à l'aide du [notebook suivant](Create-csv-files.ipynb).**

Dénormalisation retenue:

In [1]:
{
    "members":
    {
        "member_no": "1",
        "lastname": "DOUGLAS",
        "firstname": "Michael",
        "phone_number":"blabla",
 
        "location":
        {
            "country":"France",
            "region_no":"IDF",
            "city":"Paris",
            "mail_code":"75000",
            "street":"blabla"
        },
 
        "balance":
        {
            "prev_balance": "1000",
            "curr_balance": "1200"
        },
 
        "_comment":"No need to have a list, each employee have only 1 corporation",
        "corporation":
        {
            "corp_no":1,
            "corp_name":"blabla",
            "region_no":"XX"
        },
 
        "charges": [
            {
                "charge_no": "125",
                "category_no": "12",
                "category_desc": "Travel",
                "charge_dt": "2000-02-22 10:43:38",
                "charge_amount": "336.0000",
 
                "provider_no":12,
                "provider_name":"BNP Paribas",
                "provider_phone_number": "blabla",
                "payments":[1546, 1547,... , 1578]
            }
        ]
},
     "providers":
        {
            "provider_no":1,
            "provider_name":"BNP",
            "location":
            {
                "country":"France",
                "region_no":"IDF",
                "city":"Paris",
                "mail_code":"75000"
            }
        },
 
 
    "payments":
        {
            "payment_no":1546,
            "payment_dt":"2000-02-22 10:43:38",
            "payment_amt":125.00
        }
}


{'members': {'member_no': '1',
  'lastname': 'DOUGLAS',
  'firstname': 'Michael',
  'phone_number': 'blabla',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000',
   'street': 'blabla'},
  'balance': {'prev_balance': '1000', 'curr_balance': '1200'},
  '_comment': 'No need to have a list, each employee have only 1 corporation',
  'corporation': {'corp_no': 1, 'corp_name': 'blabla', 'region_no': 'XX'},
  'charge': [{'charge_no': '125',
    'category_no': '12',
    'category_desc': 'Travel',
    'charge_dt': '2000-02-22 10:43:38',
    'charge_amount': '336.0000',
    'provider_no': 12,
    'provider_name': 'BNP Paribas',
    'provider_phone_number': 'blabla',
    'payments': [1546, 1547, Ellipsis, 1578]}]},
 'providers': {'provider_no': 1,
  'provider_name': 'BNP',
  'location': {'country': 'France',
   'region_no': 'IDF',
   'city': 'Paris',
   'mail_code': '75000'}},
 'payments': {'payment_no': 1546,
  'payment_dt': '2000-02-22 10:43:38

In [2]:
import pandas as pd
import numpy as np

In [3]:
Members = pd.read_csv('Members.csv')
Corporations = pd.read_csv('Corporations.csv')
Categories = pd.read_csv('Categories.csv')
Providers = pd.read_csv('Prodivers.csv')
Charges = pd.read_csv('Charges.csv')
Statements = pd.read_csv('Statements.csv')
Payments = pd.read_csv('Payments.csv')

In [4]:
Charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
0,0,18355,60,2021-05-04,3691,151,0
1,2,5000,60,2020-07-24,9119,169,1
2,0,29818,24,2021-06-23,580,42,2
3,2,5000,36,2021-01-27,1744,138,3
4,1,5000,48,2020-07-24,766,188,4
...,...,...,...,...,...,...,...
49995,0,24383,60,2020-09-25,810,169,49995
49996,1,5000,36,2021-05-20,4205,119,49996
49997,0,5000,24,2021-06-11,654,102,49997
49998,0,5000,24,2020-11-11,600,135,49998


In [5]:
Payments

,payment_amt,due_dt,charge_no,member_no,payment_no
0,83.333333,2020-08-24,1,9119,60
1,104.166667,2020-08-24,4,766,180
2,208.333333,2020-02-24,9,5235,408
3,208.333333,2020-03-24,9,5235,409
4,208.333333,2020-04-24,9,5235,410
...,...,...,...,...,...
57644,208.333333,2020-06-08,49976,3326,1795873
57645,208.333333,2020-07-08,49976,3326,1795874
57646,208.333333,2020-08-08,49976,3326,1795875
57647,246.111111,2020-08-12,49977,8111,1795896


In [6]:
member_no = 0

charges = Charges[Charges.member_no == member_no]
charges

,Charge type,Charge amount,Duration,dt,member_no,provider_no,charge_no
13828,1,21644,48,2020-03-03,0,80,13828
27011,0,3013,48,2021-07-19,0,17,27011
29999,1,31035,36,2020-10-28,0,63,29999
38046,2,5000,60,2021-09-07,0,192,38046
45500,0,5000,24,2020-01-22,0,160,45500


In [7]:
providers_no = charges.provider_no.values
providers_no

array([ 80,  17,  63, 192, 160])

In [8]:
providers = []
for provider_no in providers_no:
    provider = Providers[Providers.provider_no == provider_no].loc[provider_no]
    providers.append(provider)
providers

[provider_no                          80
 provider_name    PNC Financial Services
 phone_number                 7587582311
 country                   United States
 region_no                            GA
 city                           Savannah
 mail_code                         31410
 street            206 Blue Marlin Drive
 corp_no                              30
 Name: 80, dtype: object,
 provider_no                    17
 provider_name       Goldman Sachs
 phone_number           4842016676
 country             United States
 region_no                      VT
 city                     Westmore
 mail_code                    5860
 street           210 Lacross Lane
 corp_no                       194
 Name: 17, dtype: object,
 provider_no                          63
 provider_name    PNC Financial Services
 phone_number                 5242901009
 country                   United States
 region_no                            AL
 city                         Montgomery
 mail_code        

In [9]:
charges_no = charges.charge_no.values
charges_no

array([13828, 27011, 29999, 38046, 45500])

In [10]:
payments = []
for charge_no in charges_no:
    payments_no = Payments[Payments.charge_no == charge_no]['payment_no'].values
    payments.append(list(payments_no))
payments

[[492000, 492001, 492002, 492003, 492004],
 [],
 [],
 [],
 [1635840, 1635841, 1635842, 1635843, 1635844, 1635845, 1635846]]

In [12]:
charges_json = []
for i in range(len(charges_no)):
    charge = charges.iloc[i-1]
    provider = providers[i]
    payments_list = payments[i]
    json = {
        "charge_no": charge.charge_no,
        "category_no": charge['Charge type'],
        #"category_desc": "Travel",
        #"charge_dt": "2000-02-22 10:43:38",
        "charge_amount": charge['Charge amount'],

        "provider_no": provider.provider_no,
        "provider_name": provider.provider_name,
        "provider_phone_number": provider.phone_number,
        "payments": payments_list
    }
    charges_json.append(json)
    
charges_json

[{'charge_no': 45500,
  'category_no': 0,
  'charge_amount': 5000,
  'provider_no': 80,
  'provider_name': 'PNC Financial Services',
  'provider_phone_number': 7587582311,
  'payments': [492000, 492001, 492002, 492003, 492004]},
 {'charge_no': 13828,
  'category_no': 1,
  'charge_amount': 21644,
  'provider_no': 17,
  'provider_name': 'Goldman Sachs',
  'provider_phone_number': 4842016676,
  'payments': []},
 {'charge_no': 27011,
  'category_no': 0,
  'charge_amount': 3013,
  'provider_no': 63,
  'provider_name': 'PNC Financial Services',
  'provider_phone_number': 5242901009,
  'payments': []},
 {'charge_no': 29999,
  'category_no': 1,
  'charge_amount': 31035,
  'provider_no': 192,
  'provider_name': 'Citigroup',
  'provider_phone_number': 695780724,
  'payments': []},
 {'charge_no': 38046,
  'category_no': 2,
  'charge_amount': 5000,
  'provider_no': 160,
  'provider_name': 'Citigroup',
  'provider_phone_number': 9711319710,
  'payments': [1635840, 1635841, 1635842, 1635843, 1635844

In [16]:
member = Members.loc[lambda x: x.member_no == member_no].loc[member_no]
member

member_no                         0
lastname                      Jones
firstname                   Brandon
phone_number             6235129717
country               United States
region_no                        AR
city                   Fayetteville
mail_code                     72701
street          17 West 26th Circle
Name: 0, dtype: object

In [19]:
member_json = {
    "member_no": member_no,
    "lastname": member.lastname,
    "firstname": member.firstname,
    "phone_number": member.phone_number,

    "location":
    {
        "country": member.country,
        "region_no": member.region_no,
        "city": member.city,
        "mail_code": member.mail_code,
        "street": member.street
    },

    #"balance":
    #{
        #"prev_balance": "1000",
        #"curr_balance": "1200"
    #},
    #"corporation":
    #{
        #"corp_no":1,
        #"corp_name":"blabla",
        #"region_no":"XX"
    #},

    "charges": charge_json
}
member_json

{'member_no': 0,
 'lastname': 'Jones',
 'firstname': 'Brandon',
 'phone_number': 6235129717,
 'location': {'country': 'United States',
  'region_no': 'AR',
  'city': 'Fayetteville',
  'mail_code': 72701,
  'street': '17 West 26th Circle'},
 'charges': [{'charge_no': 45500,
   'category_no': 0,
   'charge_amount': 5000,
   'provider_no': 80,
   'provider_name': 'PNC Financial Services',
   'provider_phone_number': 7587582311,
   'payments': [492000, 492001, 492002, 492003, 492004]},
  {'charge_no': 13828,
   'category_no': 1,
   'charge_amount': 21644,
   'provider_no': 17,
   'provider_name': 'Goldman Sachs',
   'provider_phone_number': 4842016676,
   'payments': []},
  {'charge_no': 27011,
   'category_no': 0,
   'charge_amount': 3013,
   'provider_no': 63,
   'provider_name': 'PNC Financial Services',
   'provider_phone_number': 5242901009,
   'payments': []},
  {'charge_no': 29999,
   'category_no': 1,
   'charge_amount': 31035,
   'provider_no': 192,
   'provider_name': 'Citigroup'